
# LLM + PEFT - Question Answering

Author : 정상근 (hugmanskj@gmail.com)

> 알림>
> 교육용 목적으로 제작된 코드입니다.


이 노트북에서는 Large Language Model 을 각 도메인의 데이터에 맞게 미세조정(Fine-Tuning) 하는 방법에 대해 소개하겠습니다.


저명인사들의 유명한 영어문구(English Quotes)를 [데이터셋](https://huggingface.co/datasets/Abirate/english_quotes)사용할 것이고, 이를 이용해 대규모 언어모델(LLM)을 최적화 해보겠습니다.

데이터 셋 및 몇몇 자료는 이 [사이트](https://colab.research.google.com/drive/11TwzqP8hG2cDEby5soAy_jN9DRMKDSZU?usp=sharing#scrollTo=b6fQzRl2faSn)를 참고하였습니다.

아래의 소프트웨어와 프레임워크를 사용해서 진행하겠습니다.

## 필요 소프트웨어 설치

In [1]:
#!pip install einops

## 필요한 Library Import 하기

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Big Model 을 작은 GPU 에 넣는 기법들

실용적으로 사용할 만한 테크닉이 몇 가지를 소개합니다. 계속해서 새로운 테크닉이 나오고, 사용할 수 있는 툴도 다양해지고 있기 때문에 모든 것을 소개해드리기는 힘듭니다만, Huggingface 상황에서 쉽게 적용할 수 있는 몇가지 테크닉을 소개합니다.




### [모델 로딩시 적용] 8Bit 로 Model 로딩해오기
Pretrained Model 을 로딩해올 때, load_in_8bit=True 옵션은 딥러닝 모델 훈련과 추론 과정에서 메모리 사용량을 크게 줄이는 방법입니다. 이 옵션을 활성화하면 모델의 가중치와 활성화들이 8비트 정밀도로 저장되고 처리됩니다. 이 접근법의 주요 장점과 효과는 다음과 같습니다:

- 메모리 사용량 감소: 8비트 정밀도로 모델을 로드하면 기존 32비트(float) 형식에 비해 메모리 사용량이 대폭 감소합니다. 이는 특히 GPU 메모리가 제한된 상황에서 유용하며, 더 큰 모델을 사용하거나 더 큰 배치 크기로 훈련하는 데 도움이 됩니다.
- 훈련 및 추론 속도 향상: 메모리 사용량이 줄어들면 데이터 전송 속도가 빨라지고, 연산이 더 효율적으로 이루어질 수 있습니다. 결과적으로, 모델 훈련과 추론 속도가 향상될 수 있습니다.


#### 주의사항
- 정밀도와 성능: 8비트 정밀도는 32비트보다 낮기 때문에, 때때로 모델의 성능(정확도 등)에 영향을 줄 수 있습니다. 따라서 성능에 민감한 작업에서는 8비트와 32비트 모델의 성능을 비교하는 것이 중요합니다.
- 하드웨어 호환성: 모든 하드웨어가 8비트 연산을 지원하는 것은 아닙니다. 따라서 8비트 모델을 사용하기 전에 하드웨어의 호환성을 확인하는 것이 좋습니다.

----


### [모델에 적용] Gradient Checkpointing 적용하기
huggingface의 경우 'model.gradient_checkpointing_enable()' 라는 함수를 제공합니다. 이 함수는 딥러닝 모델, 특히 메모리 집약적인 대규모 모델의 훈련 시 메모리 효율성을 크게 향상시키는 기법인 Gradient Checkpointing을 활성화합니다.

표준 backpropagation에서 그래디언트를 계산하기 위해, 신경망은 forward pass 동안의 activations (각 레이어의 출력)을 저장합니다. 이 activations은 backward pass에서 그래디언트를 계산하는 데 사용됩니다. 하지만 많은 레이어를 가진 딥 네트워크에서 이 activations을 모두 저장하는 것은 상당한 양의 메모리를 소비할 수 있습니다.

이 기법의 주요 장점과 효과는 다음과 같습니다:

- 메모리 사용량 감소: Gradient Checkpointing은 전방향 전달(forward pass) 동안의 모든 활성화(activations)를 저장하는 대신, 특정 레이어에서만 활성화를 저장하고, 필요할 때 나머지를 다시 계산합니다. 이는 전체적인 메모리 사용량을 크게 줄여줍니다.

- 대규모 모델 훈련 가능: 이 기법을 사용하면 메모리 사용량이 감소함에 따라, 기존에는 GPU 메모리 제약으로 인해 훈련하기 어려웠던 대규모 모델도 훈련할 수 있게 됩니다.

- 배치 크기 증가: 메모리 절감으로 인해 더 큰 배치 크기로 훈련을 진행할 수 있게 되며, 이는 모델의 성능과 안정성에 긍정적인 영향을 미칠 수 있습니다.


#### 주의사항
- 계산 오버헤드: 활성화를 다시 계산해야 하므로, 훈련 시간이 다소 증가할 수 있습니다. 이는 메모리 절약과 계산 시간 사이의 트레이드오프를 고려해야 합니다.
- 하이퍼파라미터 조정: Gradient Checkpointing을 사용할 때는 학습률 및 다른 하이퍼파라미터를 조정할 필요가 있을 수 있습니다.

### [모델에 적용] LoRA 적용하기

LoRA(Low-Rank Adaptation)는 특히 사전 훈련된 딥러닝 모델, 예를 들어 Transformer 기반 모델의 효율적인 미세 조정을 가능하게 하는 기술입니다. lora_config를 사용하여 LoRA를 적용하면 다음과 같은 효과를 기대할 수 있습니다:

- 효율적인 파라미터 수정: LoRA는 모델의 기존 가중치를 직접 수정하지 않고, 저차원 행렬을 도입하여 기존 가중치에 적용합니다. 이 접근법은 모델의 파라미터 수를 크게 늘리지 않으면서도 사전 훈련된 모델의 성능을 개선할 수 있습니다.

- 빠른 미세 조정 및 적응: LoRA를 적용한 모델은 새로운 데이터 또는 특정 작업에 대해 빠르게 미세 조정될 수 있습니다. 이는 전체 모델을 처음부터 훈련시키는 것보다 시간과 자원을 절약할 수 있게 해줍니다.

- 유연한 훈련과 확장성: lora_config에서는 LoRA의 다양한 매개변수(예: 저차원 행렬의 랭크)를 조정할 수 있어, 모델의 크기와 복잡도를 효과적으로 관리할 수 있습니다. 이를 통해 모델의 용량과 성능 사이에서 균형을 맞출 수 있습니다.

- 기존 가중치 보존: LoRA는 사전 훈련된 모델의 가중치를 보존하면서 새로운 데이터에 대한 학습을 가능하게 합니다. 이는 전이 학습(Transfer Learning)에서 중요한 요소로, 모델이 기존에 학습한 지식을 유지하면서 새로운 정보를 통합할 수 있게 해줍니다.

#### 주의사항
- 하이퍼파라미터 조정: LoRA의 효과는 lora_config에서 설정한 하이퍼파라미터에 크게 의존합니다. 적절한 랭크와 학습률 등을 실험을 통해 찾는 것이 중요합니다.

- 특정 모델 및 작업에 최적화: LoRA는 특히 Transformer 기반 모델에 적합합니다. 따라서 모델의 종류와 특정 작업에 따라 LoRA의 적용 방식과 효과가 달라질 수 있습니다.

----

### [훈련 시 적용] fp16 옵션 켜기
fp16=True 설정은 딥러닝 모델 훈련 시 16비트 부동소수점(FP16, Half Precision)을 사용하도록 지정하는 옵션입니다. 이 설정은 모델 훈련의 메모리 사용량을 줄이고, 훈련 속도를 높이는 데 도움이 됩니다. fp16 모드의 주요 장점과 효과는 다음과 같습니다:

- 메모리 사용량 감소: FP16을 사용하면 32비트 부동소수점(FP32, Full Precision)에 비해 메모리 사용량이 절반으로 줄어듭니다. 이는 특히 GPU 메모리가 제한된 상황에서 유용하며, 더 큰 모델을 사용하거나 더 큰 배치 크기로 훈련하는 데 도움이 됩니다.

- 훈련 및 추론 속도 향상: 메모리 대역폭이 감소하고 연산 속도가 빨라지기 때문에, 전체적인 모델 훈련과 추론 속도가 향상됩니다. FP16은 특히 GPU에서의 행렬 연산에서 더 빠른 속도를 제공합니다.

#### 주의사항
- 정밀도와 성능: FP16은 FP32보다 낮은 정밀도를 가지므로, 때때로 모델의 성능(정확도 등)에 영향을 줄 수 있습니다. 성능 저하를 최소화하기 위해, 많은 딥러닝 프레임워크들은 FP16과 FP32를 혼합하여 사용하는 Mixed Precision 훈련을 지원합니다.

- 하드웨어 호환성: 모든 GPU가 FP16 연산을 최적화하여 지원하는 것은 아닙니다. 따라서 FP16 모드를 사용하기 전에 하드웨어의 호환성을 확인하는 것이 중요합니다.
----


### [훈련 시 적용] Graident Accumulation 사용하기

gradient_accumulation_steps 설정은 딥러닝 모델 훈련 시 배치에 대한 그래디언트를 즉시 업데이트하는 대신, 여러 스텝에 걸쳐 그래디언트를 누적(accumulate)한 후에 업데이트하는 방식을 정의합니다. 이 옵션의 주요 장점과 효과는 다음과 같습니다:

- 큰 배치 크기 효과: 실제 물리적 배치 크기는 제한되어 있지만, gradient_accumulation_steps를 증가시킴으로써 더 큰 배치 크기의 효과를 낼 수 있습니다. 이는 메모리 한계 때문에 큰 배치 크기를 직접 사용할 수 없는 경우에 특히 유용합니다.

- 메모리 효율성 증가: 더 큰 배치 크기를 사용하는 것과 유사한 효과를 내면서도, 각 스텝에서 처리하는 실제 데이터 양은 작게 유지되므로 GPU 메모리 사용량을 줄일 수 있습니다.

- 모델 성능 및 안정성 개선: 더 큰 배치 크기로 인한 이점을 통해 모델의 성능이 향상되고, 훈련 과정이 더 안정적으로 진행될 수 있습니다. 큰 배치 크기는 그래디언트 업데이트의 분산을 줄여주며, 이는 일반적으로 더 안정적인 최적화 과정으로 이어집니다.

#### 주의사항
- 훈련 시간: 그래디언트 누적은 실제 업데이트가 더 드물게 일어나므로, 같은 수의 에포크를 완료하는 데 더 긴 시간이 소요될 수 있습니다.

- 하이퍼파라미터 조정: gradient_accumulation_steps를 변경하면 학습률과 같은 다른 하이퍼파라미터를 재조정할 필요가 있을 수 있습니다.

## Original Language Model 준비 및 살펴보기

### Big Model 을 작은 GPU 에 넣기

#### 8Bit 로 Model 로딩해오기

7b 모델을 Quantization 해서 다운로드 받고, 현재 머신에 업로드 합니다.

In [3]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable : {100 * trainable_params / all_param}%"
    )
    return trainable_params

In [4]:
!pip install bitsandbytes==0.46.1


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

model_id = "NousResearch/Llama-2-7b-hf"
tokenizer_id = model_id

# 8bit quantization 설정
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    bnb_8bit_use_double_quant=True,
)

# 모델 로딩
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map='auto',
    trust_remote_code=True,
    torch_dtype=torch.float16,
)

# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## LLM 의 PEFT 화 

### Parameter Frozen화

In [6]:
"""
## following parameter settings is borrowed from 
## https://github.com/huggingface/peft/issues/80
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
#model.enable_input_require_grads()

"""

'\n## following parameter settings is borrowed from \n## https://github.com/huggingface/peft/issues/80\nfor param in model.parameters():\n  param.requires_grad = False  # freeze the model - train adapters later\n  if param.ndim == 1:\n    # cast the small parameters (e.g. layernorm) to fp32 for stability\n    param.data = param.data.to(torch.float32)\n\nmodel.gradient_checkpointing_enable()  # reduce number of stored activations\n#model.enable_input_require_grads()\n\n'

In [7]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
 

### LoRA 적용

In [8]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    # Llama-2 전용 target modules
    target_modules=[
        "q_proj", "v_proj", "k_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    bias="none",

    use_rslora=False,  # RSLoRA 사용 안함
    use_dora=False,    # DoRA 사용 안함
    init_lora_weights=True,  # LoRA 가중치 초기화
)
model = get_peft_model(model, lora_config)


In [9]:
peft_p = print_trainable_parameters(model)

trainable params: 39,976,960 || all params: 6,778,392,576 || trainable : 0.589770503135875%


## Dataset 준비

In [10]:
import transformers
from datasets import load_dataset
dataset = load_dataset("Abirate/english_quotes")

English Quotes 를 datasets 라는 library 를 통해 다운받고, 전처리 할 수 있습니다.

Dataset 을 살펴보고 싶으시면 아래 링크로 가셔서 확인해 볼 수 있습니다.

[English Quotes 데이터 셋 살펴보기](https://huggingface.co/datasets/Abirate/english_quotes/viewer/default/train)

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags'],
        num_rows: 2508
    })
})

In [12]:
dataset['train']['quote'][:5]

['“Be yourself; everyone else is already taken.”',
 "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”",
 "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”",
 '“So many books, so little time.”',
 '“A room without books is like a body without a soul.”']

In [13]:
dataset['train']['author'][:5]

['Oscar Wilde',
 'Marilyn Monroe',
 'Albert Einstein',
 'Frank Zappa',
 'Marcus Tullius Cicero']

In [14]:
dataset['train']['tags'][:5]

[['be-yourself',
  'gilbert-perreira',
  'honesty',
  'inspirational',
  'misattributed-oscar-wilde',
  'quote-investigator'],
 ['best', 'life', 'love', 'mistakes', 'out-of-control', 'truth', 'worst'],
 ['human-nature',
  'humor',
  'infinity',
  'philosophy',
  'science',
  'stupidity',
  'universe'],
 ['books', 'humor'],
 ['books', 'simile', 'soul']]

우리는 이번 실습에서 Tag 정보를 넣으면, 그 tag에 걸맞는 Quote가 생성되도록 해볼 겁니다.

즉, 사용 시나리오는

> be-yourself ===> "~~~~~~~~~~~~~~~~~~~~~~"

이렇게 관심 tag를 넣어주고 "===>" 기호를 넣어주면 그 뒤를 LM 이 생성하는 방식으로 Q/A를 풀어낼 겁니다.



In [15]:
def merge_cols(example):
    example["prediction"] = ', '.join(example["tags"]) + " ===> " + example["quote"]
    return example

dataset['train'] = dataset['train'].map(merge_cols) # <-- 모든 문장에 대해 처리해 줍니다.
dataset['train']["prediction"][5:7]

["ataraxy, be-yourself, confidence, fitting-in, individuality, misattributed-dr-seuss, those-who-matter ===> “Be who you are and say what you feel, because those who mind don't matter, and those who matter don't mind.”",
 "dance, heaven, hurt, inspirational, life, love, sing ===> “You've gotta dance like there's nobody watching,Love like you'll never be hurt,Sing like there's nobody listening,And live like it's heaven on earth.”"]

In [16]:
dataset['train'][5]

{'quote': "“Be who you are and say what you feel, because those who mind don't matter, and those who matter don't mind.”",
 'author': 'Bernard M. Baruch',
 'tags': ['ataraxy',
  'be-yourself',
  'confidence',
  'fitting-in',
  'individuality',
  'misattributed-dr-seuss',
  'those-who-matter'],
 'prediction': "ataraxy, be-yourself, confidence, fitting-in, individuality, misattributed-dr-seuss, those-who-matter ===> “Be who you are and say what you feel, because those who mind don't matter, and those who matter don't mind.”"}

In [17]:
dataset = dataset.map(
                        lambda x: tokenizer(x['prediction']),
                        batched=True
                     )

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags', 'prediction', 'input_ids', 'attention_mask'],
        num_rows: 2508
    })
})

## Fine-Tuning 하기

제대로 된 실험을 위해서는 최소 몇 시간 이상의 훈련이 필요합니다. 이번 실습에서는 적당한 수의 Step 만큼만 훈련 을 진행해보겠습니다.

In [19]:
train_args=transformers.TrainingArguments(
        per_device_train_batch_size=2,

        warmup_steps=100,
        max_steps=400,
        #max_steps=20,

        learning_rate=2e-4,

        # -- peft -- #
        gradient_accumulation_steps=6,
        fp16=True, # or bf16=True
        # ---------- #

        logging_steps=20,
        output_dir='outputs'
    )

tokenizer.pad_token = tokenizer.eos_token #
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset['train'],
    args=train_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [20]:
trainer.train()

TrainOutput(global_step=400, training_loss=0.987444965839386, metrics={'train_runtime': 954.9754, 'train_samples_per_second': 5.026, 'train_steps_per_second': 0.419, 'total_flos': 1.5184447802474496e+16, 'train_loss': 0.987444965839386, 'epoch': 1.9138755980861244})

### Model 저장

In [21]:
pretrained_model_path = 'llm_peft_qa_v1'  # it will be directory
model.save_pretrained(pretrained_model_path)

## Inference 해보기

In [22]:
# 1) outputs 에 저장된 파일을 불러오고
# 2) 불러온 모델에 대해 다시 PEFT 시킨다.
lora_config = LoraConfig.from_pretrained(pretrained_model_path)
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'NousResearch/Llama-2-7b-hf' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [23]:
example_batch = tokenizer("individuality, majority, minority, reflect, wisdom ===> ", return_tensors='pt')

# 입력 텐서를 모델과 같은 device로 이동
example_batch = {k: v.to(model.device) for k, v in example_batch.items()}

# 최신 autocast 사용법 적용
with torch.amp.autocast('cuda'):
    output_tokens = model.generate(
        **example_batch, 
        max_new_tokens=50,
        use_cache=False,  # gradient checkpointing 사용 시 필요
        do_sample=True,
        temperature=0.7,
    )

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))




 individuality, majority, minority, reflect, wisdom ===> 0 Comments
It is the duty of the minority to be the conscience of the majority.
majority, minority, wisdom ===> 0 Comments
majority, minority, wisdom ===> 0 Comments
major


In [24]:
def generate_text_safe(model, tokenizer, prompt, max_new_tokens=50):
    """안전한 텍스트 생성 함수"""
    
    # 모델의 device 확인
    device = next(model.parameters()).device
    print(f"모델 device: {device}")
    
    # 입력 토큰화 및 device 이동
    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # 생성 설정
    generation_config = {
        'max_new_tokens': max_new_tokens,
        'use_cache': False,  # gradient checkpointing 호환성
        'do_sample': True,
        'temperature': 0.7,
        'top_p': 0.9,
        'pad_token_id': tokenizer.eos_token_id,
    }
    
    # 텍스트 생성
    with torch.amp.autocast('cuda'):
        output_tokens = model.generate(**inputs, **generation_config)
    
    # 결과 디코딩
    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    
    return generated_text

# 사용 예시
result = generate_text_safe(
    model, 
    tokenizer, 
    "individuality, majority, minority, reflect, wisdom ===> ",
    max_new_tokens=50
)
print(f"\n생성된 텍스트:\n{result}")

모델 device: cuda:0

생성된 텍스트:
individuality, majority, minority, reflect, wisdom ===> 116:44, July 11, 2014
116:44, July 11, 2014: Individuality, Majority, Minority, Reflect, Wis
